In [2]:
import numpy as np
import os
import collections

In [1]:
sents = []
sent_words = []
with open('../1998-01-2003版-带音.txt', 'r', encoding='gbk') as f:
    for line in f.readlines():
        sent = ''
        sent_word = []
        words = line.strip().split('  ')[1:]
        for word in words:
            sent += (word.split('/')[0].strip() + ' ')
            sent_word.append(word.split('/')[0].strip())
        sents.append(sent)
        sent_words.append(sent_word)

In [3]:
words_set = []
for sent_word in sent_words:
    words_set += ''.join(sent_word)
all_words = words_set
words_set = list(set(words_set))

In [4]:
with open('./unigram.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(dict(collections.Counter(all_words))))
with open('./bigram.json', 'w', encoding='utf-8') as f:
    bigrams = []
    for word1, word2 in zip(all_words[:-1], all_words[1:]):
        bigrams.append(word1+word2)
    f.write(json.dumps(dict(collections.Counter(bigrams))))

In [5]:
marked_sent_words = []
for sent_word in sent_words:
    marks = []
    for word in sent_word:
        if len(word) == 1:
            marks.append('s')
        elif len(word) == 2:
            marks.append('be')
        elif len(word) > 2:
            marks.append('b'+('m'*(len(word)-2))+'e')
    assert(len(sent_word)==len(marks))
    marked_sent_words.append(marks)

In [6]:
import collections
all_words = []
all_marks = []
for words, marks in zip(sent_words, marked_sent_words):
    all_words += words
    all_marks += marks
all_words = ''.join(all_words)
all_marks = ''.join(all_marks)
print(len(all_words), len(all_marks))
status = ['b', 'm', 'e', 's']
word_marks = {s: [] for s in status}
for mark, word in zip(all_marks, all_words):
    word_marks[mark].append(word)
for s in word_marks:
    l = len(word_marks[s])
    word_marks[s] = dict(collections.Counter(word_marks[s]))
    word_marks[s] = {k: np.log(word_marks[s][k] / l) for k in word_marks[s]}

1915301 1915301


In [7]:
# import json
# with open('./word_marks_count.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(word_marks))
import json
with open('./word_marks_count.json', 'r', encoding='utf-8') as f:
    word_marks = json.loads(f.read())

In [14]:
class Word_cutter():
    def __init__(self):
        if os.path.exists('../1998-01-2003版-带音.txt'):
            self.sents = []
            self.sent_words = []
            with open('../1998-01-2003版-带音.txt', 'r', encoding='gbk') as f:
                for line in f.readlines():
                    sent = ''
                    sent_word = []
                    words = line.strip().split('  ')[1:]
                    for word in words:
                        sent += (word.split('/')[0].strip() + ' ')
                        sent_word.append(word.split('/')[0].strip())
                    self.sents.append(sent)
                    self.sent_words.append(sent_word)
            self.emit_mat = self.get_emit_mat()
            self.log_init_status, self.log_trans_mat = self.get_init_status_and_trans_mat()
                    
    def get_emit_mat(self):
        if os.path.exists('./word_marks_count.json'):
            print('emit exits')
            with open('./word_marks_count.json', 'r', encoding='utf-8') as f:
                di = json.loads(f.read())
            emit_mat = {}
            for word_mark in di:
                for word in di[word_mark]:
                    if word in emit_mat:
                        emit_mat[word][word_mark] = di[word_mark][word]
                    else:
                        emit_mat[word] = {'b': 0., 'm': 0., 'e': 0., 's': 0.}
            return emit_mat
        words_set = []
        for sent_word in self.sent_words:
            words_set += ''.join(sent_word)
        words_set = list(set(words_set))

        self.marked_sent_words = []
        for sent_word in self.sent_words:
            marks = []
            for word in sent_word:
                if len(word) == 1:
                    marks.append('s')
                elif len(word) == 2:
                    marks.append('be')
                elif len(word) > 2:
                    marks.append('b'+('m'*(len(word)-2))+'e')
            assert(len(sent_word)==len(marks))
            self.marked_sent_words.append(marks)

        all_words = []
        all_marks = []
        for words, marks in zip(self.sent_words, self.marked_sent_words):
            all_words += words
            all_marks += marks
        all_words = ''.join(all_words)
        all_marks = ''.join(all_marks)
        status = ['b', 'm', 'e', 's']
        word_marks = {s: [] for s in status}
        for mark, word in zip(all_marks, all_words):
            word_marks[mark].append(word)
        for s in word_marks:
            l = len(word_marks[s])
            word_marks[s] = dict(collections.Counter(word_marks[s]))
            word_marks[s] = {k: np.log(word_marks[s][k] / l) for k in word_marks[s]}

        marks_per_word = {}
        for word_mark in word_marks:
            for word in word_marks[word_mark]:
                if word in marks_per_word:
                    marks_per_word[word][word_mark] = word_marks[word_mark][word]
                else:
                    marks_per_word[word] = {'b': 0., 'm': 0., 'e': 0., 's': 0.}
        return marks_per_word
    
    def get_init_status_and_trans_mat(self):
        if (os.path.exists('./log_init_status.json') and os.path.exists('./log_trans_mat.json')):
            print('init exits')
            with open('./log_init_status.json', 'r', encoding='utf-8') as f:
                log_init_mat = json.loads(f.read())
            with open('./log_trans_mat.json', 'r', encoding='utf-8') as f:
                log_trans_mat = json.loads(f.read())
            return log_init_mat, log_trans_mat
        init_status = np.zeros((4, 1))
        trans_mat = np.zeros((4, 4))
        status2ind = {'b': 0, 'm': 1, 'e': 2, 's': 3}
        temp_count = {'b': 0., 's': 0.}
        #初始化初始状态
        for m in self.marked_sent_words:
            if len(m) == 0:
                continue
            if m[0][0] == 'b':
                temp_count['b'] += 1.
            elif m[0][0] == 's':
                temp_count['s'] += 1.
        init_status[0, 0] = temp_count['b'] / (temp_count['b'] + temp_count['s'])
        init_status[3, 0] = temp_count['s'] / (temp_count['b'] + temp_count['s'])
        #构造转移矩阵
        all_marked_seq = ''
        for m in self.marked_sent_words:
            if len(m) == 0:
                continue
            else:
                all_marked_seq += ''.join(m)
        for mark1 ,mark2 in zip(all_marked_seq[:-1], all_marked_seq[1:]):
            trans_mat[status2ind[mark1], status2ind[mark2]] += 1.
        trans_mat /= np.sum(trans_mat)
        log_trans_mat = np.log(trans_mat)
        log_init_status = np.log(init_status)
        return log_init_status, np.array(log_trans_mat)
    
    def cut_words(self, txt, emit_mat=None, log_init_status=None, log_trans_mat=None):
        if emit_mat == None:
            emit_mat = self.get_emit_mat()
        if log_init_status == None or log_trans_mat == None:
            log_init_status, log_trans_mat = self.get_init_status_and_trans_mat()
            print(log_trans_mat)
        log_trans_mat = np.array(log_trans_mat)
        print(log_init_status)
        weights = np.ones((4, len(txt))) * -9999.
        status = ['b', 'm', 'e', 's']
        weights[:, 0] = np.squeeze(log_init_status) + np.array(list(emit_mat[txt[0]].values())).astype(np.float64)
        #初始化第一个字的向量
        path = -np.ones((4, len(txt)))
        for i, word in enumerate(txt[:-1]):
            ind = i + 1
            for j in range(4):
                for k in range(4):
                    temp = weights[k, ind-1] + log_trans_mat[k, j] + float(emit_mat[txt[ind]][status[j]])
                    if temp > weights[j, ind]:
                        weights[j, ind] = temp
                        path[j, ind] = k
        endE, endS = weights[2, weights.shape[1]-1], weights[3, weights.shape[1]-1]
        inx = 0
        if endE > endS:
            inx = 2
        else:
            inx = 3
        result = []
        for i in reversed(range(len(txt))):
            result.append(inx)
            inx = path[inx, i].astype(int)
        
        marks = reversed(result)
        cis = []
        temp = ''
        for mark, letter in zip(marks, txt):
            if (mark == 0) or (mark == 1):
                temp += letter
            elif mark == 2:
                temp += letter
                cis.append(temp)
                temp = ''
            elif mark == 3:
                temp = ''
                cis.append(letter)
        return ' '.join(cis)

In [15]:
txt = '年来贵州省各级党委和政府把扶贫开发工作作为农村中心任务来抓取得很大成绩贫困人口大量减少贫困状况明显改善'
cutter = Word_cutter()

emit exits
init exits


In [16]:
output = cutter.cut_words(txt)


emit exits
init exits
[[-inf, -0.51082562376599, -0.916290731874155, -inf], [-inf, -0.33344856811948514, -1.2603623820268226, -inf], [-0.5897149736854513, -inf, -inf, -0.8085250474669937], [-0.7211965654669841, -inf, -inf, -0.6658631448798212]]
[-0.26268660809250016, -inf, -inf, -1.4652633398537678]


In [17]:
import tkinter as tk
import tkinter.messagebox

def getPoetry():
    txt = title_in.get()
    output = ''.join([item[0] for item in cutter.cut_words(txt)])
    print(output, type(output))
    t.delete('1.0', '2.0')
    t.insert('1.0', output)


# 庆春， 腊梅香， 清朝， 菩萨， 浣溪沙
win = tk.Tk()
var = tk.StringVar()
var.set("")
title_in = tk.Entry(win, textvariable  = var, width=45)
title_in.pack()
t = tk.Text(win, )

t.pack()

button_get = tk.Button(win, text="开始分词", command=getPoetry)
button_get.pack()
win.mainloop()

emit exits
init exits
[[-inf, -0.51082562376599, -0.916290731874155, -inf], [-inf, -0.33344856811948514, -1.2603623820268226, -inf], [-0.5897149736854513, -inf, -inf, -0.8085250474669937], [-0.7211965654669841, -inf, -inf, -0.6658631448798212]]
[-0.26268660809250016, -inf, -inf, -1.4652633398537678]
今天 天气 真好 啊 <class 'str'>
emit exits
init exits
[[-inf, -0.51082562376599, -0.916290731874155, -inf], [-inf, -0.33344856811948514, -1.2603623820268226, -inf], [-0.5897149736854513, -inf, -inf, -0.8085250474669937], [-0.7211965654669841, -inf, -inf, -0.6658631448798212]]
[-0.26268660809250016, -inf, -inf, -1.4652633398537678]
新一 届中 共中 央 政治局 常委 将在 人民 大会 堂同 中外 记者 见 面， 欢迎 各位 记者 参加 <class 'str'>
emit exits
init exits
[[-inf, -0.51082562376599, -0.916290731874155, -inf], [-inf, -0.33344856811948514, -1.2603623820268226, -inf], [-0.5897149736854513, -inf, -inf, -0.8085250474669937], [-0.7211965654669841, -inf, -inf, -0.6658631448798212]]
[-0.26268660809250016, -inf, -inf, -1.4652633398537678]
从梁 家河